In [ ]:
!pip install glove-python-binary

     |████████████████████████████████| 948 kB 25.4 MB/s 


In [ ]:
import io
import os
import nltk
import re
import string
import numpy as np
import pandas as pd 

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from bs4 import BeautifulSoup  
from nltk.corpus import wordnet, stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from snowballstemmer import TurkishStemmer
from nltk.util import ngrams

import glove 
from scipy import spatial
from glove import Corpus, Glove
from multiprocessing import Pool
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data import TensorDataset, DataLoader

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving deneme1016_scraped_data.csv to deneme1016_scraped_data (1).csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['deneme1016_scraped_data.csv']))

In [ ]:
# Exploratory Data Analysis 
df.head()
df.columns
print('length of data is', len(df))
df. shape
df.info()
df.dtypes
np.sum(df.isnull().any(axis=1))

length of data is 20160
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20160 entries, 0 to 20159
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stars    20160 non-null  object
 1   comment  20160 non-null  object
dtypes: object(2)
memory usage: 315.1+ KB


0

In [ ]:
# Getting stars from stars column
stars = df["stars"]
new_stars = []
def star_list(x):
    x = list(x)
    for i in range(len(x)):
        new_stars.append(x[i][-3:-2])
    return new_stars
    
stars_no = star_list(stars)
stars_column = pd.Series(stars_no)
df['stars'] = stars_column
df.head()

,stars,comment
0,1,\n\n\n\n\n\n\n\n \n \n Bu ürünü aldığımda...
1,5,"\n\n\n\n\n\n\n\n \n \n Fiyatı da uygun, k..."
2,5,\n\n\n\n\n\n\n\n \n \n Son kullanma tarih...
3,5,\n\n\n\n\n\n\n\n \n \n Earl grey aromali ...
4,5,\n\n\n\n\n\n\n\n \n \n Bir çay bağımlısı ...


In [ ]:
# Returns sentiment value based on the overall ratings
def generate_sentiment(row):
    if row['stars'] == '3' :
        val = 'Neutral'
    elif row['stars'] == '1' or row['stars'] == '2':
        val = 'Negative'
    elif row['stars'] == '4' or row['stars'] == '5':
        val = 'Positive'
    else:
        val = -1
    return val

df['sentiment'] = df.apply(generate_sentiment, axis=1)
df['sentiment'].value_counts()
df.head()

,stars,comment,sentiment
0,1,\n\n\n\n\n\n\n\n \n \n Bu ürünü aldığımda...,Negative
1,5,"\n\n\n\n\n\n\n\n \n \n Fiyatı da uygun, k...",Positive
2,5,\n\n\n\n\n\n\n\n \n \n Son kullanma tarih...,Positive
3,5,\n\n\n\n\n\n\n\n \n \n Earl grey aromali ...,Positive
4,5,\n\n\n\n\n\n\n\n \n \n Bir çay bağımlısı ...,Positive


In [ ]:
df['stars'].value_counts(normalize=True)

5    0.614931
4    0.140575
3    0.106994
1    0.079663
2    0.057837
Name: stars, dtype: float64

Preprocessing

In [ ]:
# Clean Text
def cleantext(raw_text, remove_stopwords=True, stemming=True, split_text = False):
    text1 = BeautifulSoup(raw_text, 'lxml').get_text()  # remove html
    text2 = re.sub('[0123456789,/\.!?:‘’()"]', '', text1) # remove non-character
    words = text2.lower().split() # convert to lower case 
    if remove_stopwords:
      stop_word_list = nltk.corpus.stopwords.words('turkish') # removing stopwords
      words = [w for w in words if not w in stop_word_list]
    if stemming == True: # stemming Turkish words
      turkStem = TurkishStemmer()
      words = [turkStem.stemWord(word) for word in words]
    if split_text == True: # split text
      return (words)
    return(" ".join(words))

In [ ]:
reviews = df['comment']
X_cleaned = []
for review in reviews:
   X_cleaned.append(cleantext(review))

In [ ]:
# Split review text into parsed sentences using NLTK's punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def Text_to_Sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(cleantext(raw_sentence, remove_stopwords, split_text=True))
    return sentences

In [ ]:
sentences = []
for review in X_cleaned:
  sentences += Text_to_Sentences(review, tokenizer)

In [ ]:
def track_vocab(sentences, verbose =  True):
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
                
    return vocab

In [ ]:
# count the occurrence of all words in the data
vocab_count = track_vocab(sentences)
print({k: vocab_count[k] for k in list(vocab_count)[:5]})

{'ür': 4817, 'al': 6002, 'ber': 424, 'tadı': 2391, 'bir': 26783}


In [ ]:
def create_lookup_tables(vocab_count):
    
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(vocab_count, key=vocab_count.get, reverse=True)
    # create vocab_to_int dictionary
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)

vocab_to_int, int_to_vocab = create_lookup_tables(vocab_count)

In [ ]:
# encode the data
sentence_ints = []
for sentence in sentences:
    sentence_ints.append([vocab_to_int[word] for word in sentence])

In [ ]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))

# print tokens in first review
print('Tokenized review: \n', sentence_ints[:1])

Unique words:  1610
Tokenized review: 
 [[8, 4, 348, 30, 0, 492, 2, 493, 44, 494, 495, 11, 95, 257, 340, 496, 497, 11, 250, 498, 499, 231, 307, 500, 140, 501, 183, 502, 38, 503, 50, 284, 232, 504, 345, 280, 231, 232, 505, 506, 507, 308, 73, 127, 175, 508, 0, 346, 509, 510, 45]]


In [ ]:
def pad_features(sentences_token, seq_length):
    # getting the correct rows x cols shape
    features = np.zeros((len(sentences_token), seq_length), dtype=int)

    # for each title, I grab that title and 
    for i, row in enumerate(sentences_token):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

# pad the titles
seq_length = 50
features = pad_features(sentence_ints, seq_length)

In [ ]:
labels = df['sentiment'] 
print(labels)

0        Negative
1        Positive
2        Positive
3        Positive
4        Positive
           ...   
20155    Positive
20156    Positive
20157    Positive
20158    Positive
20159    Positive
Name: sentiment, Length: 20160, dtype: object


In [ ]:
# Replace labels with integers
dict_labels = {'Negative':0, 'Neutral':1, 'Positive': 2}
labels = pd.Series(labels).replace(dict_labels)
labels = labels.to_numpy(copy = True)

Word2Vec Embeddings



In [ ]:
model = Word2Vec(sentences = sentences, size = 300, sg = 1, window = 3, min_count = 1, iter = 10, workers = Pool()._processes)
model.init_sims(replace = True)
model.save('word2vec_model')

print("Number of words in the vocabulary list : %d \n" %len(model.wv.index2word))
print("Show first 10 words in the vocabulary list: \n", model.wv.index2word[0:10])
# Load trained Word2Vec model
model = Word2Vec.load('word2vec_model')

# Get Word2Vec embedding matrix
word2vec_embed_matrix = model.wv.syn0 
print("Shape of embedding matrix : ", word2vec_embed_matrix.shape)
print(word2vec_embed_matrix)

Number of words in the vocabulary list : 1610 

Show first 10 words in the vocabulary list: 
 ['bir', 'kitap', 'ol', 'amazo', 'al', 'gel', 'kargo', 'kahve', 'ür', 'fiyat']
Shape of embedding matrix :  (1610, 300)
[[-0.06131336 -0.01756017 -0.04114821 ...  0.07660954 -0.01107388
  -0.02477599]
 [ 0.01701378 -0.01741518 -0.02642088 ...  0.09859084 -0.0417672
  -0.10511018]
 [ 0.03217573 -0.03930807 -0.05874291 ... -0.03861464 -0.0320034
   0.00843136]
 ...
 [-0.00347647  0.04661679 -0.14094475 ...  0.03727179 -0.05411803
  -0.09462743]
 [ 0.06006055 -0.0681548  -0.09083479 ...  0.00026503  0.0009223
  -0.03580105]
 [ 0.0764944  -0.07224622 -0.06080306 ...  0.01526372 -0.01336818
  -0.03159595]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# Create Word2Vec Dictionary 
word2vec_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    word2vec_dict[key] = model.wv[key]

Glove Embeddings

In [ ]:
# Create Corpus instance
corpus = Corpus()
# training the corpus to generate the co-occurence matrix which is used in GloVe
corpus.fit(sentences, window = 3) 
#creating a Glove object which will use the matrix created in the above lines
glove = Glove(no_components = 300, learning_rate = 0.05)
# Train model
glove.fit(matrix = corpus.matrix, epochs = 30, no_threads = Pool()._processes, verbose = True)
# Save and load model
glove.add_dictionary(corpus.dictionary)
glove.save('glove_model')

Performing 30 training epochs with 2 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [ ]:
glove_model = glove.load("glove_model")
glove_embed_matrix = glove_model.word_vectors
print("Vector Representation for the word alışveriş in Glove:")
np.array(glove_model.word_vectors[glove_model.dictionary['alışveriş']]).reshape(1,300)

Vector Representation for the word alışveriş in Glove:


array([[-3.14627044e-02,  7.94729168e-02, -1.54854273e-01,
        -6.87758843e-03, -1.00538478e-01,  4.01113893e-02,
         1.29170042e-01,  3.12598159e-02, -1.66763067e-01,
         9.06033295e-02, -2.70750069e-05, -1.45088151e-01,
         5.58917038e-02, -8.52732500e-02,  1.25663393e-02,
        -1.29933471e-01,  7.41891042e-02,  8.13972306e-02,
        -6.55204516e-02,  1.35513398e-01, -9.92604186e-02,
         1.57667148e-01,  1.66576146e-01,  9.87070127e-02,
         7.47653958e-02, -1.84725478e-01, -2.95330495e-03,
         9.67495717e-02,  5.20756294e-02, -1.18573324e-01,
        -5.67743345e-02, -9.64702836e-02, -1.23140686e-01,
        -3.93832980e-03,  4.91984458e-02,  5.62260471e-03,
         4.62605275e-02,  1.96246717e-02,  1.80406849e-01,
        -9.75186135e-02,  6.20987737e-02,  3.75416419e-02,
         1.28134980e-02,  2.78962646e-02, -1.08896986e-01,
         1.30806558e-01, -9.98335092e-02, -1.64244363e-01,
         1.11729163e-01, -6.77652915e-02,  1.08143766e-0

FastText Embeddings

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!cp fasttext ../
%cd ..
!pip install fasttext
import fasttext 

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 3930 (delta 29), reused 70 (delta 29), pack-reused 3854
Receiving objects: 100% (3930/3930), 8.33 MiB | 33.33 MiB/s, done.
Resolving deltas: 100% (2446/2446), done.
/content/fastText
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++11 -march=native -O3 

In [ ]:
# Download, extract and load Fasttext word embedding model
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.bin.gz

--2022-05-22 11:02:12--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4506977940 (4.2G) [application/octet-stream]
Saving to: ‘cc.tr.300.bin.gz’

cc.tr.300.bin.gz    100%[===================>]   4.20G  45.8MB/s    in 1m 42s  

2022-05-22 11:03:55 (42.0 MB/s) - ‘cc.tr.300.bin.gz’ saved [4506977940/4506977940]



In [ ]:
# Download, extract and load Fasttext word embedding vectors
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz
# !gunzip cc.tr.300.vec.gz

--2022-05-22 08:23:59--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1261500728 (1.2G) [binary/octet-stream]
Saving to: ‘cc.tr.300.vec.gz’

cc.tr.300.vec.gz    100%[===================>]   1.17G  38.4MB/s    in 32s     

2022-05-22 08:24:32 (37.8 MB/s) - ‘cc.tr.300.vec.gz’ saved [1261500728/1261500728]



In [ ]:
import fasttext.util
fasttext.util.download_model('tr', if_exists='ignore')
ft_model = fasttext.load_model('cc.tr.300.bin')
ft_model.save_model('ft_model')

In [ ]:
ft_model.get_word_vector("büşra")
ft_model.get_nearest_neighbors('zeki')

[(0.7558695673942566, 'zeki.'),
 (0.7393233776092529, 'zekii'),
 (0.7049357891082764, 'yetenekli'),
 (0.6554639935493469, 'becerikli'),
 (0.6511648893356323, 'zekisi'),
 (0.650030255317688, 'zekiler'),
 (0.6492676138877869, 'zekidir'),
 (0.6472271084785461, 'çalışkan'),
 (0.647098183631897, 'meziyetli'),
 (0.6430851221084595, 'zekiyse')]

In [ ]:
# def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
    return data

In [ ]:
# if __name__ == '__main__':
    ft_embed_matrix = load_vectors('cc.tr.300.vec')

Creating Embedding Layer

In [ ]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

For Word2vec Weights Matrix

In [ ]:
sorted_vocab = sorted(vocab_count, key=vocab_count.get, reverse=True)
target_vocab = sorted_vocab
emb_dim = 300

matrix_len = len(target_vocab)
word2vec_weights_matrix = np.zeros((matrix_len, emb_dim))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        word2vec_weights_matrix[i] = word2vec_dict[word]
        words_found += 1
    except KeyError:
        word2vec_weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

For Glove Weights Matrix

In [ ]:
sorted_vocab = sorted(vocab_count, key=vocab_count.get, reverse=True)
target_vocab = sorted_vocab
emb_dim = 300

matrix_len = len(target_vocab)
glove_weights_matrix = np.zeros((matrix_len, emb_dim))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        glove_weights_matrix[i] = glove_model.dictionary[word]
        words_found += 1
    except KeyError:
        glove_weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

For FastText Weight Matrix

In [ ]:
sorted_vocab = sorted(vocab_count, key=vocab_count.get, reverse=True)
target_vocab = sorted_vocab
emb_dim = 300

matrix_len = len(target_vocab)
ft_weights_matrix = np.zeros((matrix_len, emb_dim))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        ft_weights_matrix[i] = ft_model.get_word_vector(word)
        words_found += 1
    except KeyError:
        ft_weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

Hyperparameters

For Word2vec Embeddings

In [ ]:
# hyperparameters 
weights_matrix = word2vec_weights_matrix
output_size = 3
hidden_dim = 128
drop_prob = 0.1

For Glove Embeddings

In [ ]:
# hyperparameters 
weights_matrix = glove_weights_matrix 
output_size = 3
hidden_dim = 32
drop_prob = 0.5

For FastText Embeddings

In [ ]:
# hyperparameters 
weights_matrix = ft_weights_matrix 
output_size = 3
hidden_dim = 64
drop_prob = 0.5

NEURAL NETWORKS MODELS

RNN Network Architecture with LSTM

RNN Architecture 1

In [ ]:
# The RNN model that will be used to perform classification
class RNN1(nn.Module):

    def __init__(self, weights_matrix, output_size, hidden_dim, drop_prob):

        super(RNN1, self).__init__()
        
        # embedding layers
        self.embedding, self.num_embeddings, self.embeddings_size = create_emb_layer(weights_matrix, True)
        # embedding dropout
        self.dropout = nn.Dropout2d(drop_prob)
        
        # First lstm and GRU layers
        self.lstm1 = nn.LSTM(self.embeddings_size, hidden_dim, batch_first=True, bidirectional=True)
        self.gru1 = nn.GRU(hidden_dim * 2, hidden_dim, bidirectional=True, batch_first=True)
        
        # Second lstm and GRU layers
        self.lstm2 = nn.LSTM(hidden_dim * 2, hidden_dim, batch_first=True, bidirectional=True)
        self.gru2 = nn.GRU(hidden_dim * 2, hidden_dim, bidirectional=True, batch_first=True)
        
        # linear
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.out = nn.Linear(hidden_dim * 2, output_size)
        
        self.softmax = nn.Softmax(dim = 1)
        

    def forward(self, x):

        batch_size = x.size(0)
        # embedding output
        x = x.long()
        embeds = self.embedding(x)
        embeds = torch.squeeze(torch.unsqueeze(embeds, 0))
        
        # lstm, and gru outputs
        lstm_out, _ = self.lstm1(embeds)
        # slice lstm_out to just get output of last element of the input sequence
#        lstm_out = lstm_out[:, -1]
        gru_out, _ = self.gru1(lstm_out)
        gru_out = gru_out.view(batch_size, -1, hidden_dim * 2)
        lstm_out, _ = self.lstm2(gru_out)
        # slice lstm_out to just get output of last element of the input sequence
        lstm_out = lstm_out[:, -1]
        gru_out, _ = self.gru2(lstm_out)
        
        # linear outputs
        fc_out = self.softmax(gru_out)
        final_out = self.out(fc_out)
    
        return final_out

if __name__ == '__main__':
  
  # Configuration options
  k_folds = 3
  num_epochs = 2
  loss_function = nn.CrossEntropyLoss()
  
  # For fold results
  results = {}
  
  # Set fixed random number seed
  torch.manual_seed(42)
  
  # Prepare dataset by concatenating Train/Test part; we split later.
  train_X, test_X, train_y, test_y = train_test_split(features, labels, 
                                                            test_size=0.2, 
                                                            random_state=42, shuffle=True,
                                                            stratify=labels)
  
  dataset_train_part = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
  dataset_test_part = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))
  dataset = ConcatDataset([dataset_train_part, dataset_test_part])
  
  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
    # Print
    print(f'FOLD {fold}')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    
    # Init the neural network and optimizer
    network = RNN1(weights_matrix, output_size, hidden_dim, drop_prob)
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
    
    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):
        
        # Get inputs
        inputs, targets = data
        
        optimizer.zero_grad()
        outputs = network(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 50 == 49:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 50))
            current_loss = 0.0
        
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')
    
    # Saving the model
    save_path = f'./rnn1-ft-model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)


        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        output_labels = outputs.argmax(dim=1).detach()
        scores = precision_recall_fscore_support(targets, output_labels, average='weighted', labels=np.unique(output_labels))
        
      # Print scores
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      results[fold] = 100.0 * (correct / total)
      print('Precision : ', scores[0], 'Recall : ', scores[1], 'F-score : ', scores[2])  

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')

FOLD 0
Starting epoch 1
Loss after mini-batch    50: 1.107
Loss after mini-batch   100: 1.095
Loss after mini-batch   150: 1.080
Loss after mini-batch   200: 1.070
Loss after mini-batch   250: 1.058
Loss after mini-batch   300: 1.052
Loss after mini-batch   350: 1.045
Loss after mini-batch   400: 1.040
Loss after mini-batch   450: 1.029
Loss after mini-batch   500: 1.023
Loss after mini-batch   550: 1.020
Loss after mini-batch   600: 1.009
Loss after mini-batch   650: 1.006
Loss after mini-batch   700: 0.999
Loss after mini-batch   750: 0.979
Loss after mini-batch   800: 0.987
Loss after mini-batch   850: 0.979
Loss after mini-batch   900: 0.970
Loss after mini-batch   950: 0.975
Loss after mini-batch  1000: 0.960
Loss after mini-batch  1050: 0.965
Loss after mini-batch  1100: 0.951
Loss after mini-batch  1150: 0.956
Loss after mini-batch  1200: 0.954
Loss after mini-batch  1250: 0.949
Loss after mini-batch  1300: 0.917
Starting epoch 2
Loss after mini-batch    50: 0.913
Loss after min

RNN Architecture 2

In [ ]:
class RNN2(nn.Module):

    def __init__(self, weights_matrix, output_size, hidden_dim, drop_prob):

        super(RNN2, self).__init__()
        
        # embedding layers
        self.embedding, self.num_embeddings, self.embeddings_size = create_emb_layer(weights_matrix, True)
        
        # embedding dropout
        self.dropout = nn.Dropout2d(drop_prob)
        
        # First lstm and GRU layers
        self.lstm1 = nn.LSTM(self.embeddings_size, hidden_dim, batch_first=True, bidirectional=True)
        self.gru1 = nn.GRU(hidden_dim * 2, hidden_dim, bidirectional=True, batch_first=True)
        
        # Second lstm and GRU layers
        self.lstm2 = nn.LSTM(hidden_dim * 2, hidden_dim, batch_first=True, bidirectional=True)
        self.gru2 = nn.GRU(hidden_dim * 2, hidden_dim, bidirectional=True, batch_first=True)

        # Third lstm and GRU layers
        self.lstm3 = nn.LSTM(hidden_dim * 2, hidden_dim, batch_first=True, bidirectional=True)
        self.gru3 = nn.GRU(hidden_dim * 2, hidden_dim, bidirectional=True, batch_first=True)
        
        # linear
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.out = nn.Linear(hidden_dim * 2, output_size)
        
        self.softmax = nn.Softmax(dim=1)
        

    def forward(self, x):

        batch_size = x.size(0)
        # embedding output
        x = x.long()
        embeds = self.embedding(x)
        embeds = torch.squeeze(torch.unsqueeze(embeds, 0))
        
        # lstm, and gru outputs
        lstm_out, _ = self.lstm1(embeds)
        # slice lstm_out to just get output of last element of the input sequence
        lstm_out = lstm_out[:, -1]
        gru_out, _ = self.gru1(lstm_out)
        gru_out = gru_out.view(batch_size, -1, hidden_dim * 2)
        lstm_out, _ = self.lstm2(gru_out)
        # slice lstm_out to just get output of last element of the input sequence
        lstm_out = lstm_out[:, -1]
        gru_out, _ = self.gru2(lstm_out)
        lstm_out, _ = self.lstm3(gru_out)
        # slice lstm_out to just get output of last element of the input sequence
  #      lstm_out = lstm_out[:, -2]
        gru_out, _ = self.gru3(lstm_out)
        # linear outputs
        fc_out = self.softmax(gru_out)
        final_out = self.out(fc_out)
    
        return final_out


if __name__ == '__main__':
  
  # Configuration options
  k_folds = 3
  num_epochs = 2
  loss_function = nn.CrossEntropyLoss()
  
  # For fold results
  results = {}
  
  # Set fixed random number seed
  torch.manual_seed(42)
  
  # Prepare dataset by concatenating Train/Test part; we split later.
  train_X, test_X, train_y, test_y = train_test_split(features, labels, 
                                                            test_size=0.2, 
                                                            random_state=42, shuffle=True,
                                                            stratify=labels)
  
  dataset_train_part = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
  dataset_test_part = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))
  dataset = ConcatDataset([dataset_train_part, dataset_test_part])
  
  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
    # Print
    print(f'FOLD {fold}')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    
    # Init the neural network and optimizer
    network = RNN2(weights_matrix, output_size, hidden_dim, drop_prob)
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
    
    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):
        
        # Get inputs
        inputs, targets = data
        
        optimizer.zero_grad()
        outputs = network(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 50 == 49:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 50))
            current_loss = 0.0
        
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')
    
    # Saving the model
    save_path = f'./rnn2-ft-model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)


        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        output_labels = outputs.argmax(dim=1).detach()
        scores = precision_recall_fscore_support(targets, output_labels, average='weighted', labels=np.unique(output_labels))
        
      # Print scores
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      results[fold] = 100.0 * (correct / total)
      print('Precision : ', scores[0], 'Recall : ', scores[1], 'F-score : ', scores[2])  

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')

FOLD 0
Starting epoch 1
Loss after mini-batch    50: 1.091
Loss after mini-batch   100: 1.077
Loss after mini-batch   150: 1.059
Loss after mini-batch   200: 1.046
Loss after mini-batch   250: 1.043
Loss after mini-batch   300: 1.036
Loss after mini-batch   350: 1.027
Loss after mini-batch   400: 1.017
Loss after mini-batch   450: 1.014
Loss after mini-batch   500: 1.008
Loss after mini-batch   550: 1.001
Loss after mini-batch   600: 0.998
Loss after mini-batch   650: 0.990
Loss after mini-batch   700: 0.974
Loss after mini-batch   750: 0.977
Loss after mini-batch   800: 0.968
Loss after mini-batch   850: 0.962
Loss after mini-batch   900: 0.960
Loss after mini-batch   950: 0.968
Loss after mini-batch  1000: 0.948
Loss after mini-batch  1050: 0.950
Loss after mini-batch  1100: 0.928
Loss after mini-batch  1150: 0.918
Loss after mini-batch  1200: 0.918
Loss after mini-batch  1250: 0.930
Loss after mini-batch  1300: 0.916
Starting epoch 2
Loss after mini-batch    50: 0.902
Loss after min

Convolutional Neural Networks (CNNs)

Hyperparameters

For Word2Vec Embeddings

In [ ]:
# hyperparameters 
weights_matrix = word2vec_weights_matrix
num_classes = 3
hidden_dim = 64
drop_prob = 0.5
padding_idx = 1

For Glove Embeddings

In [ ]:
# hyperparameters 
weights_matrix = glove_weights_matrix 
output_size = 3
hidden_dim = 32
drop_prob = 0.5
padding_idx = 1

For FastText Embeddings

In [ ]:
# hyperparameters 
weights_matrix = ft_weights_matrix
num_classes = 3
hidden_dim = 128
drop_prob = 0.5
padding_idx = 1

CNN Architecture 1

In [ ]:
class CNN1(nn.Module):
    def __init__(self, weights_matrix, drop_prob, padding_idx, num_classes):
        super(CNN1, self).__init__()

        self.embedding, self.num_embeddings, self.embeddings_size = create_emb_layer(weights_matrix, True)
        self.dropout = nn.Dropout(drop_prob)

        # conv2d
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=256,
                               kernel_size=(3, self.embeddings_size),
                               stride=1) # output: [batch_size, output_channels, max_len - 3 + 1, 1]
        self.conv2 = nn.Conv2d(in_channels=1,
                               out_channels=128,
                               kernel_size=(3, 256),
                               stride=1) # output: [batch_size, output_channels, , 1]

        # linear
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32, num_classes)

        # log soft max
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, inputs):

        batch_size = inputs.size(0)
        embedded = self.embedding(inputs) # [batch_size, len, embedding_size]
        embedded = self.dropout(embedded)

        # conv
        conv1_output = self.conv1(embedded.unsqueeze(1))
        conv1_output = self.softmax (conv1_output)
        conv1_output = conv1_output.squeeze(3).transpose(1, 2) # [batch_size, len - stride + 1, output_channels]

        conv2_output = self.conv2(conv1_output.unsqueeze(1))
        conv2_output = self.softmax (conv2_output)

        # max pool [batch_size, output_channels, 1, 1]
        max_pool_output = F.max_pool2d(conv2_output, kernel_size=(conv2_output.shape[2], 1))

        # [batch_size, output_channels] out_channels
        fc1_input = max_pool_output.squeeze() #[batch_size, output_channels]

        fc1_output = self.fc1(fc1_input)
        fc2_output = self.fc2(fc1_output)

        # softmax
        output = self.softmax (fc2_output)

        return output

if __name__ == '__main__':
  
  # Configuration options
  k_folds = 3
  num_epochs = 2
  loss_function = nn.CrossEntropyLoss()
  
  # For fold results
  results = {}
  
  # Set fixed random number seed
  torch.manual_seed(42)
  
  # Prepare dataset by concatenating Train/Test part; we split later.
  train_X, test_X, train_y, test_y = train_test_split(features, labels, 
                                                            test_size=0.2, 
                                                            random_state=42, shuffle=True,
                                                            stratify=labels)
  
  dataset_train_part = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
  dataset_test_part = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))
  dataset = ConcatDataset([dataset_train_part, dataset_test_part])
  
  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
    # Print
    print(f'FOLD {fold}')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    
    # Init the neural network and optimizer
    network = CNN1(weights_matrix, drop_prob, padding_idx, num_classes)
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
    
    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):
        
        # Get inputs
        inputs, targets = data
        
        optimizer.zero_grad()
        outputs = network(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 50 == 49:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 50))
            current_loss = 0.0
        
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')
    
    # Saving the model
    save_path = f'./cnn1-ft-model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)


        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        output_labels = outputs.argmax(dim=1).detach()
        scores = precision_recall_fscore_support(targets, output_labels, average='weighted', labels=np.unique(output_labels))
        
      # Print scores
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      results[fold] = 100.0 * (correct / total)
      print('Precision : ', scores[0], 'Recall : ', scores[1], 'F-score : ', scores[2])  

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')

FOLD 0
Starting epoch 1
Loss after mini-batch    50: 1.070
Loss after mini-batch   100: 1.061
Loss after mini-batch   150: 1.048
Loss after mini-batch   200: 1.021
Loss after mini-batch   250: 1.006
Loss after mini-batch   300: 0.993
Loss after mini-batch   350: 0.970
Loss after mini-batch   400: 0.935
Loss after mini-batch   450: 0.932
Loss after mini-batch   500: 0.918
Loss after mini-batch   550: 0.875
Loss after mini-batch   600: 0.878
Loss after mini-batch   650: 0.854
Loss after mini-batch   700: 0.872
Loss after mini-batch   750: 0.867
Loss after mini-batch   800: 0.854
Loss after mini-batch   850: 0.825
Loss after mini-batch   900: 0.816
Loss after mini-batch   950: 0.778
Loss after mini-batch  1000: 0.796
Loss after mini-batch  1050: 0.800
Loss after mini-batch  1100: 0.831
Loss after mini-batch  1150: 0.804
Loss after mini-batch  1200: 0.798
Loss after mini-batch  1250: 0.823
Loss after mini-batch  1300: 0.805
Starting epoch 2
Loss after mini-batch    50: 0.823
Loss after min

CNN Architecture 2

In [ ]:
class CNN2(nn.Module):
    def __init__(self, weights_matrix, drop_prob, padding_idx, num_classes):
        super(CNN2, self).__init__()

        self.embedding, self.num_embeddings, self.embeddings_size = create_emb_layer(weights_matrix, True)
        self.dropout = nn.Dropout(drop_prob)

        # conv2d
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=256,
                               kernel_size=(3, self.embeddings_size),
                               stride=1) # output: [batch_size, output_channels, max_len - 3 + 1, 1]
        self.conv2 = nn.Conv2d(in_channels=1,
                               out_channels=128,
                               kernel_size=(3, 256),
                               stride=1) # output: [batch_size, output_channels, , 1]
        self.conv3 = nn.Conv2d(in_channels=1,
                               out_channels=64,
                               kernel_size=(3, 128),
                               stride=1) 
        
        # linear
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32, num_classes)

        # log soft max
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, inputs):

        batch_size = inputs.size(0)
        embedded = self.embedding(inputs) # [batch_size, len, embedding_size]
        embedded = self.dropout(embedded)

        # conv
        conv1_output = self.conv1(embedded.unsqueeze(1))
        conv1_output =  self.softmax(conv1_output)
        conv1_output = conv1_output.squeeze(3).transpose(1, 2) # [batch_size, len - stride + 1, output_channels]

        conv2_output = self.conv2(conv1_output.unsqueeze(1))
        conv2_output =  self.softmax(conv2_output)

        # max pool [batch_size, output_channels, 1, 1]
        max_pool_output = F.max_pool2d(conv2_output, kernel_size=(conv2_output.shape[2], 1))

        # [batch_size, output_channels] out_channels
        fc1_input = max_pool_output.squeeze() #[batch_size, output_channels]

        fc1_output = self.fc1(fc1_input)
        fc2_output = self.fc2(fc1_output)

        # softmax
        output =  self.softmax(fc2_output)

        return output

if __name__ == '__main__':
  
  # Configuration options
  k_folds = 3
  num_epochs = 2
  loss_function = nn.CrossEntropyLoss()
  
  # For fold results
  results = {}
  
  # Set fixed random number seed
  torch.manual_seed(42)
  
  # Prepare dataset by concatenating Train/Test part; we split later.
  train_X, test_X, train_y, test_y = train_test_split(features, labels, 
                                                            test_size=0.2, 
                                                            random_state=42, shuffle=True,
                                                            stratify=labels)
  
  dataset_train_part = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
  dataset_test_part = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))
  dataset = ConcatDataset([dataset_train_part, dataset_test_part])
  
  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
    # Print
    print(f'FOLD {fold}')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    
    # Init the neural network and optimizer
    network = CNN2(weights_matrix, drop_prob, padding_idx, num_classes)
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
    
    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):
        
        # Get inputs
        inputs, targets = data
        
        optimizer.zero_grad()
        outputs = network(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 50 == 49:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 50))
            current_loss = 0.0
        
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')
    
    # Saving the model
    save_path = f'./cnn-ft-model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)


        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        output_labels = outputs.argmax(dim=1).detach()
        scores = precision_recall_fscore_support(targets, output_labels, average='weighted', labels=np.unique(output_labels))
        
      # Print scores
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      results[fold] = 100.0 * (correct / total)
      print('Precision : ', scores[0], 'Recall : ', scores[1], 'F-score : ', scores[2])  

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')

FOLD 0
Starting epoch 1


RuntimeError: ignored

CNN 3

In [ ]:
class CNN1(nn.Module):
    def __init__(self, weights_matrix, drop_prob, padding_idx, num_classes):
        super(CNN1, self).__init__()

        self.embedding, self.num_embeddings, self.embeddings_size = create_emb_layer(weights_matrix, True)
        self.dropout = nn.Dropout(drop_prob)

        # conv2d
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=256,
                               kernel_size=(3, self.embeddings_size),
                               stride=2) # output: [batch_size, output_channels, max_len - 3 + 1, 1]
        self.conv2 = nn.Conv2d(in_channels=1,
                               out_channels=128,
                               kernel_size=(3, 256),
                               stride=2) # output: [batch_size, output_channels, , 1]

        # linear
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, num_classes)

        # log soft max
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, inputs):

        batch_size = inputs.size(0)
        embedded = self.embedding(inputs) # [batch_size, len, embedding_size]
        embedded = self.dropout(embedded)

        # conv
        conv1_output = self.conv1(embedded.unsqueeze(1))
        conv1_output = self.softmax (conv1_output)
        conv1_output = conv1_output.squeeze(3).transpose(1, 2) # [batch_size, len - stride + 1, output_channels]

        conv2_output = self.conv2(conv1_output.unsqueeze(1))
        conv2_output = self.softmax (conv2_output)

        # max pool [batch_size, output_channels, 1, 1]
        max_pool_output = F.max_pool2d(conv2_output, kernel_size=(conv2_output.shape[2], 1))

        # [batch_size, output_channels] out_channels
        fc1_input = max_pool_output.squeeze() #[batch_size, output_channels]

        fc1_output = self.fc1(fc1_input)
        fc2_output = self.fc2(fc1_output)

        # softmax
        output = self.softmax (fc2_output)

        return output

if __name__ == '__main__':
  
  # Configuration options
  k_folds = 3
  num_epochs = 2
  loss_function = nn.CrossEntropyLoss()
  
  # For fold results
  results = {}
  
  # Set fixed random number seed
  torch.manual_seed(42)
  
  # Prepare dataset by concatenating Train/Test part; we split later.
  train_X, test_X, train_y, test_y = train_test_split(features, labels, 
                                                            test_size=0.2, 
                                                            random_state=42, shuffle=True,
                                                            stratify=labels)
  
  dataset_train_part = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
  dataset_test_part = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))
  dataset = ConcatDataset([dataset_train_part, dataset_test_part])
  
  # Define the K-fold Cross Validator
  kfold = KFold(n_splits=k_folds, shuffle=True)

  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
    # Print
    print(f'FOLD {fold}')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=10, sampler=test_subsampler)
    
    # Init the neural network and optimizer
    network = CNN1(weights_matrix, drop_prob, padding_idx, num_classes)
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
    
    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

      # Print epoch
      print(f'Starting epoch {epoch+1}')

      # Set current loss value
      current_loss = 0

      # Iterate over the DataLoader for training data
      for i, data in enumerate(trainloader, 0):
        
        # Get inputs
        inputs, targets = data
        
        optimizer.zero_grad()
        outputs = network(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # Print statistics
        current_loss += loss.item()
        if i % 50 == 49:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 50))
            current_loss = 0.0
        
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')
    
    # Saving the model
    save_path = f'./cnn3-ft-model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():

      # Iterate over the test data and generate predictions
      for i, data in enumerate(testloader, 0):

        # Get inputs
        inputs, targets = data

        # Generate outputs
        outputs = network(inputs)


        # Set total and correct
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        output_labels = outputs.argmax(dim=1).detach()
        scores = precision_recall_fscore_support(targets, output_labels, average='weighted', labels=np.unique(output_labels))
        
      # Print scores
      print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
      results[fold] = 100.0 * (correct / total)
      print('Precision : ', scores[0], 'Recall : ', scores[1], 'F-score : ', scores[2])  

  # Print fold results
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')

FOLD 0
Starting epoch 1
Loss after mini-batch    50: 1.074
Loss after mini-batch   100: 1.057
Loss after mini-batch   150: 1.040
Loss after mini-batch   200: 1.024
Loss after mini-batch   250: 1.009
Loss after mini-batch   300: 0.972
Loss after mini-batch   350: 0.961
Loss after mini-batch   400: 0.939
Loss after mini-batch   450: 0.937
Loss after mini-batch   500: 0.902
Loss after mini-batch   550: 0.899
Loss after mini-batch   600: 0.874
Loss after mini-batch   650: 0.849
Loss after mini-batch   700: 0.831
Loss after mini-batch   750: 0.835
Loss after mini-batch   800: 0.838
Loss after mini-batch   850: 0.828
Loss after mini-batch   900: 0.799
Loss after mini-batch   950: 0.812
Loss after mini-batch  1000: 0.845
Loss after mini-batch  1050: 0.800
Loss after mini-batch  1100: 0.833
Loss after mini-batch  1150: 0.817
Loss after mini-batch  1200: 0.821
Loss after mini-batch  1250: 0.814
Loss after mini-batch  1300: 0.798
Starting epoch 2
Loss after mini-batch    50: 0.781
Loss after min